In [1]:
import re 
import datetime
import pandas as pd

import urllib.request as urllib_request
from urllib.error import URLError, HTTPError

from utils.helpers import *
from utils.functions import *
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', None)

In [2]:
URL = 'https://fiis.com.br/lista-de-fundos-imobiliarios/'
soup = create_bsoup_object(URL)

In [3]:
tickers = get_tickers(soup)

In [4]:
dataset = create_fiis_dataframe(tickers[0:10])

ABCP11
AFHI11
AFOF11
AIEC11
ALMI11
ALZR11
ANCR11B
APTO11
ARCT11
ARRI11


In [5]:
COLUMNS_LIST = ['ultimo_rendimento',
                'valor_patrimonial_por_cota',
                'numero_cotas',
                'numero_cotistas',
                'cotacao',
                'min_52_weeks',
                'max_52_weeks']
dataset = convert_string_to_float(dataset, COLUMNS_LIST)
dataset = convert_percentages_to_float(dataset, ['dividend_yield', 'return_12_months'])
dataset = break_tipo_FII(dataset)
dataset['registro_CVM'] = dataset['registro_CVM'].apply(lambda x: str(convert_to_datetime(x)))
#dataset["p_vp"] = (dataset['cotacao'] / dataset['valor_patrimonial_por_cota']).replace(np.inf, np.nan)
dataset['date'] = str(datetime.date.today() - datetime.timedelta(days=1))
dataset.head()

,ticker,administrador,dividend_yield,ultimo_rendimento,patrimonio_liquido,valor_patrimonial_por_cota,telefone,email,site,name,tipo_ANBIMA,registro_CVM,numero_cotas,numero_cotistas,CNPJ,cotacao,min_52_weeks,max_52_weeks,return_12_months,tipo,subtipo,date
0,ABCP11,Rio Bravo Investimentos,0.0090,0.65,"R$1,12 B",91.83,(11) 3509-6600,[email protected],www.riobravo.com.br,FII ABC IMOB,Renda Gestão Passiva,1996-05-15 00:00:00,12203832.0,19040.0,01.201.140/0001-90,72.56,60.01,78.27,0.0404,Tijolo,Shoppings,2022-05-27
1,AFHI11,BTG Pactual,0.0130,1.35,"R$172,46 M",96.47,21 3262-9600,[email protected],www.btgpactual.com,FII AFHI CRI,Títulos e Valores Mobiliários Gestão Ativa,2020-10-20 00:00:00,1787671.0,7700.0,36.642.293/0001-58,103.80,88.00,103.99,0.1320,Papel,CRIs,2022-05-27
2,AFOF11,BRL Trust DTVM,0.0129,1.14,"R$70,55 M",102.07,(11)31330350,[email protected],www.brltrust.com.br,FII ALIANZFF,Títulos e Valores Mobiliários Gestão Ativa,2020-01-22 00:00:00,691192.0,2584.0,34.847.063/0001-08,88.03,75.05,300.00,0.3626,Papel,Fundo de Fundos,2022-05-27
3,AIEC11,MAF DTVM,0.0090,0.72,"R$491,83 M",101.93,3223-7700,[email protected],www.modal.com.br,FII AUTONOMY,Renda Gestão Ativa,2020-08-24 00:00:00,4824987.0,6762.0,35.765.826/0001-26,79.98,67.50,88.00,0.0870,Tijolo,Escritórios,2022-05-27
4,ALMI11,BTG Pactual,0.0004,0.37,"R$250,49 M",2253.05,21 3262-9600,[email protected],www.btgpactual.com,FII TORRE AL,Renda Gestão Passiva,2006-01-03 00:00:00,111177.0,3114.0,07.122.725/0001-00,930.00,830.00,1300.00,0.2119,Tijolo,Escritórios,2022-05-27


In [12]:
URL = 'https://fiis.com.br/ifix/'
df_ifix = create_ifix_dataframe(URL)
COLUMNS_LIST = ['peso']
df_ifix = convert_percentages_to_float(df_ifix, COLUMNS_LIST)
df_ifix['date'] = str(datetime.date.today())
df_ifix.head()

,ticker,peso,date
0,KNIP11,0.06933,2022-05-28
1,KNCR11,0.03734,2022-05-28
2,HGLG11,0.03440,2022-05-28
3,IRDM11,0.03319,2022-05-28
4,KNRI11,0.03092,2022-05-28


## DynamoDB

### Sending the fiis dataframe

In [8]:
import boto3
import json
from decimal import Decimal

In [33]:
ticker_data_dict = dataset.to_dict(orient='records')

In [34]:
dynamodb_resource = boto3.resource("dynamodb")

In [35]:
batch_write(dynamodb_resource, "fiis", ticker_data_dict)

True

In [36]:
table = dynamodb_resource.Table("fiis")
response = table.scan()
df = pd.DataFrame(response['Items'])
df.shape

(359, 22)

### Sending the ifix dataframe

In [10]:
dynamodb_resource = boto3.resource("dynamodb")
ifix_data_dict = dataset.to_dict(orient='records')
batch_write(dynamodb_resource, "fiis", ifix_data_dict)

True